# Description of the two-step analysis

Author:<br>
Nora Poel (1,2)<br>
(1) CNRS/UGA/IRD/G-INP, IGE, Grenoble, France<br>
(2) University of Potsdam, Institute for Computer Science, Potsdam, Germany

The aim of this notebook is to describe the preparation and execution of the two-step analysis. The two-step analysis was developed specifically in preparation of the SWOT mission. It performs a data assimilation update with an observation vector that is augmented by **gradient observations** and with a corresponding observation error covariance matrix $\mathbf{R^+}$. The analysis will be run with an implementation of an Ensemble Kalman Filter ([SESAM](http://pp.ige-grenoble.fr/pageperso/brankarj/SESAM/)).

In the Ensemble Kalman Filter the pdf of the **state vector** is represented by an ensemble (sample) of members (states). In the two-step analysis the state vector consists of 6 variables all projected on different grids:<br>
ssh provided by a numerical model on gridT<br>
ssh projected with the SWOT simulator on gridSWOT<br>
first derivation along track calculated on gridD1a<br>
first derivation across track calculated on gridD1c<br>
second derivation along track calculated on gridD2a<br>
second derivation across track calculated on gridD2c<br>

In a first step we generate an ensemble of numerical model outputs that is afterwards extended by the other variables. For the development of the analysis process and the data preparation tools, a dataset consisting of 474 outputs (with a frequency of one day) from the NATL60 simulation between june 2012 and october 2013 for the OSMOSIS region was provided. In the following, the dataset will be addressed as ```alldata.nc```.

In a second step we generate the **observation vector** for the analysis based on the true state of the system. As true state we consider day 100 from the initial dataset. We then extend the observation vector with first and second derivations along and across track.

## 1. Generation of background information
As background information for the analysis an ensemble is generated from alldata.nc using the bash script [make_summer_ensemble.ksh](data_prep/make_summer_ensemble.ksh).<br>
The generated ensemble is named<br>

`ENS0060.nc.bas`<br>

and consists of 60 members, named as followed: <br>

`vctgridT<nnnn>.nc`

with `<nnnn>` ranging from 0001 to 0060. `gridT` describes the grid of the model output.<br>
(**Note**: for naming see [SESAM](http://pp.ige-grenoble.fr/pageperso/brankarj/SESAM/) (--> SESAM data types) as there are restrictions for ensemble and member names from the analysis tool SESAM.)

## 2. Generation of the true state
As true state we consider day 100 from the initial dataset. It is extracted from `alldata.nc` using the bash command:<br>

```ncks -d time_counter,100,100 alldata.nc xt_0060_gridT.nc```

## 3. Generation of SWOT-like observations
### 3.1 Extend ensemble
For each member of the ensemble `ENS0060.nc.bas` we use the SWOT simulator to project ssh on a SWOT-like grid.
The SWOT simulator was created in preparation of the SWOT mission to simulate SWOT-like observations. To extend the ensemble we only need the projection of the ssh from the model grid on the SWOT-like grid (variable name in the file generated by the simulator: `ssh_model`). For the generation of the R-matrix in section 5 we also need the realization of the expected observation errors (variable name in the file generated by the simulator: `ssh_obs`). For this work, a single SWOT pass (pass 182) that crosses the OSMOSIS region from the northwest corner to the south (center) was chosen to represent the SWOT-like grid and observation. Normally, the simulator generates all passes that cross the input region. To generate only one single pass, we let the simulator generate (almost) all passes and corresponding grids for the first member of the ensemble (see step i - iii below) and then delete all grids except of the one of the desired pass. Following steps v and vi the simulator will then only produce a pass on the provided grid.<br>
(**Note:** The gap between the two swaths is filled by the simulator.)

i) parameter file for the simulator:<br>
[params_pass182_nogap.py](data_prep/params_pass182_nogap.py)

ii) prepare input directory (input/ENS0060.nc.bas); add [list_of_file.txt](data_prep/list_of_file.txt) and [list_tmp.txt](data_prep/list_tmp.txt) in the same directory
- list_of_file.txt should consist of the following two lines:
        vctgridT0001.nc
        vctgridT0001.nc
- list_tmp.txt should consist of the following two lines:
        vctgridTNB.nc
        vctgridTNB.nc
        
iii) create output directory (output/OBS0060)

iv) run the simulator for the first member of the ensemble to generate the grid of pass182 (verify that `makegrid=TRUE`):<br> 
`swotsimulator params_pass182_nogap.py`

v) delete all files except `swot292_grid_p182.nc` and `swot292_gridnadir_p182.nc` in output directory and change `makegrid` from TRUE to FALSE in `params_pass182_nogap.py` (in the following only pass 182 will be created)

vi) run [create_obs_ens60.ksh](data_prep/create_obs_ens60.ksh) to call the simulator for each member of ensemble (except first member)

### 3.2 Observations of x-true
We create several observations from the true state to chose one of them for the subsequent analysis. Observations are created for the same pass (pass 182) as used to extend the ensemble. We can either create observations with the 20 km gap between the two swaths or without (gap filled by the simulator).

i) **with gap** use [params_pass182_withgap.py](data_prep/params_pass182_withgap.py)<br>
   **without gap** use [params_pass182_nogap.py](data_prep/params_pass182_nogap.py)
   
ii) prepare input directory (input/xt_0060); add true state (`xt_0060_gridT.nc`) and [list_of_file.txt](data_prep/obs/list_of_file.txt) to directory
- list_of_file.txt should consist of the following two lines:
        xt_0060_gridT.nc
        xt_0060_gridT.nc

iii) create output directory (output/p182_from_xt_0060); copy `swot292_grid_p182.nc` and `swot292_gridnadir_p182.nc` in output directory

iv) adjust parameter file: `makegrid=FALSE` and set right input and output directory paths

v) run [create_p182_from_xt_0060.ksh](data_prep/create_p182_from_xt_0060.ksh); adjust number of observations that will be produced and name of parameter file (with/without gap) in the script.

## 4. Processing SWOT simulator outputs

### 4.1 Processing the ensemble

i) for each members of the observation-ensemble (`OBS0060`, generated in 3.1) convert variables lon,lat,ssh_model,ssh_obs to real numbers using:<br>
[convert_vct_to_realnb.ksh](data_prep/convert_vct_to_realnb.ksh)
        
ii) process the observation-ensemble (`OBS0060`) using [20180418-np-prepare-analysis.ipynb](20180418-np-prepare-analysis.ipynb) (section 1)
- remove double nadir in from each member of the observation-ensemble (`OBS0060`)
- denoise ssh_obs to later generate R and
- augment ssh_model with its derivatives


### 4.2 Processing an observation

i) chose one observation simulated from x-true (here we chose: `obs_p182_SWOT0004.nc`)
        
ii) convert ssh_obs,lon,lat to real numbers: (probably not necessary because SWOTdenoise rewrite the data with real numbers)

        ncap -O -s "ssh_obs=(ssh_obs*1.0);lon=(lon*1.0);lat=(lat*1.0)" obs_p182_SWOT0004.nc obs_p182_SWOT0004.nc
     
iii) process `obs_p182_SWOT0004.nc` using [20180418-np-prepare-analysis.ipynb](20180418-np-prepare-analysis.ipynb) (section 2)
- remove the double nadir,
- denoise observation and 
- extend the observation

## 5. Generation of R

i) estimate parameters for the R-matrix and generate a R-matrix for each (gradient) observation using [20180418-np-parametrization-R.ipynb](20180418-np-parametrization-R.ipynb).


## 6. Perform the two-step analysis
We need to split the analysis in two subsequent analysis. This is, because there is no implementation in SESAM to interpolate gradient observations on the model grid (gridT) (= there is no implementation of the observation operator $\mathbf{H}$ for gradient observations). In the first analysis we are using all five (gradient) observations (ssh, first and second derivation along and across track) and their corresponding R matrixes to perform an update on each grid. The update that is performed on gridT shows an updated area but in a region that is spacially not correlated with the location of the observation. The update that is performed on the observation grid (gridSWOT) is spatially not correlated to the model grid but contains the updated ssh. With a second analysis we then project the updated ssh from gridSWOT on the model grid. We thereby assign a low standard deviation to the observation (here: 0.0001) so that the observation has a high certainty in the analysis. 

### 6.1 First analysis
#### 6.1.1 Prepare the analysis directory
The directory for the first analysis should contain:

i) the extended `ENS0060.nc.bas`; unite background-ensemble (`ENS0060.nc.bas` containing gridT) and processed observation-ensemble (`OBS0060` containing gridSWOT, gridD1a, gridD1c, gridD2a, gridD2c) in `ENS0060.nc.bas`

ii) all (gradient) observations; the file type need to be changed from `.nc` to `.ncdta` (e.g. `mv rmdc_obs_p182_SWOT0004_denoised_D1a.nc rmdc_obs_p182_SWOT0004_denoised_D1a.ncdta`), here:<br>

   `rmdc_obs_p182_SWOT0004_denoised_D1a.ncdta`<br>
    `rmdc_obs_p182_SWOT0004_denoised_D1c.ncdta`<br>
    `rmdc_obs_p182_SWOT0004_denoised_D2a.ncdta`<br>
    `rmdc_obs_p182_SWOT0004_denoised_D2c.ncdta`<br>
    `rmdc_obs_p182_SWOT0004_denoised_SWOT.ncdta`
    
iii) five R-matrixes:<br>

   `R_gridSWOT.nc`<br>
    `R_gridD1a.nc`<br>
    `R_gridD1c.nc`<br>
    `R_gridD2a.nc`<br>
    `R_gridD2a.nc`
    
iv) one mask for each grid type, create e.g. as followed:<br>

cp ENS0060.nc.bas/vctgridSWOT0001.nc mask_gridT.nc<br>
cp obs_p182_SWOT0007_denoised_SWOT.ncdta mask_gridSWOT.nc<br>
cp obs_p182_SWOT0007_denoised_D1a.ncdta mask_gridD1a.nc<br>
cp obs_p182_SWOT0007_denoised_D1c.ncdta mask_gridD1c.nc<br>
cp obs_p182_SWOT0007_denoised_D2a.ncdta mask_gridD2a.nc<br>
cp obs_p182_SWOT0007_denoised_D2c.ncdta mask_gridD2c.nc

(**Note**: Masks need to consist the same variable as the corresponding data files. All (gradient) observation masks must be converted to real numbers. The $\_$FillValue must be denoted in the masks. This is the case if they are produced as above.)

v) a domain localization configuration; here [L10localization.cfg](data_prep/L10localization.cfg), consisting of 4 lines:<br>
1 1 <br>
30 30<br>
10 10<br>
end<br>

vi) the configuration file for SESAM [sesamlist1](data_prep/sesamlist1)<br>
(**Note**: to be renamed as "sesamlist")

#### 6.1.2 Run analysis 1
To run the analysis use [run_analysis1_L10_p182_0060.ksh](data_prep/run_analysis1_L10_p182_0060.ksh).<br>
The script will perform the following steps:<br>
1. remove previous localization files if exist and create localization files
2. remove previous analysis results and create an output folder for the analysis
3. run the analysis

### 6.2 Second Analysis
#### 6.2.1 Prepare the analysis directory
The directory of the second analysis should contain:

i) `ENS0060.nc.bas` containing only gridT

ii) the result of analysis 1 on gridSWOT (here: xa_ana1_gridSWOT.nc)

iii) mask_gridT.nc from analysis 1 directory

iv) the configuration file for SESAM [sesamlist2](data_prep/sesamlist2)<br>
(**Note**: to be renamed as sesamlist)

#### 6.2.2 Run analysis 2
To run the analysis use [run_analysis2_L10_p182_0060.ksh](data_prep/run_analysis2_L10_p182_0060.ksh).<br>
The script will perform the following steps:<br>
1. generation of R for gridT using mask_gridT.nc as input
2. rename the observation
3. rename the mask according the specifications in sesamlist
4. adjust the longitude of the observation: The output of the first analysis stores longitude from 0° to 360° whereas in gridT files (background model) longitude ranges uses the notation from 0° to 180° with a minus sign in eastward and plus sign is westward direction. The scale of the result of analysis 1 will be adapted by the script.
5. perform the observation management
6. remove previous localization files if exist and create localization files
7. remove previous analysis results and create an output folder for the analysis
8. run the analysis

## 7. Additional SESAM commands

i) Compute the ensemble mean and standard deviation (for all grids in ensemble)<br>
`sesam -mode oper -incfg list_ENS0060.cfg -outvar xf_#.nc -typeoper mean`<br>
`sesam -mode oper -incfg list_ENS0060.cfg -outvar stdf_#.nc -typeoper std`